In [3]:
# PaddleSeg git download
!git clone https://gitee.com/paddlepaddle/PaddleSeg.git --depth=1 ./work

Cloning into './work'...
remote: Enumerating objects: 1140, done.
remote: Counting objects: 100% (1140/1140), done.
remote: Compressing objects: 100% (944/944), done.
remote: Total 1140 (delta 217), reused 807 (delta 131), pack-reused 0
Receiving objects: 100% (1140/1140), 44.45 MiB | 4.95 MiB/s, done.
Resolving deltas: 100% (217/217), done.
Checking connectivity... done.


# 1. Project preparation
PaddleSeg installation
Model and Data downloading

In [8]:
# PaddleSeg installation
%cd ~/work/
#!pip install -r requirements.txt -i https://mirror.baidu.com/pypi/simple/
!pip install paddleseg

/home/aistudio/work
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 245kB 6.5MB/s eta 0:00:01


In [ ]:
# prepare the dataset#
!unzip  -d ~/work/  ~/data/data56097/PaddleSeg.zip

In [11]:
# downlowd the pre-trained model and unzip
%cd ~/work/PaddleSeg/
!python pretrained_model/download_model.py unet_bn_coco

/home/aistudio/work/PaddleSeg
[==================================================] 100.00%
Uncompress unet_coco_v3.tgz
[==================================================] 100.00%
Pretrained Model download success!


# 2. Model training with PaddleSeg

there are two errors when running the original code written with Paddle1.8, 
one is about the static graph. the error message was quite clear and can be easily cleared by following the recommendation in the error message.
the other is more tricky. It is about the API save_op_compatible_info. It was for internal test and was never exposed to the public as in the paddle website and doc. I am not able to figure out the purpose of the api, even after contacting the supporting team. However, simply disabling the whole save_infer_program() function is a good solution to me and the result looks right.

In [2]:
# model training
%cd ~/work/PaddleSeg02/
!pwd

# 指定GPU卡号（以0号卡为例）
!export CUDA_VISIBLE_DEVICES=0
# 训练
#!python train.py --cfg configs/unet/unet_optic.yaml \
!python pdseg/train.py --cfg configs/unet_optic.yaml \
                      --use_gpu \
                      --do_eval \
                      --use_vdl \
                      --vdl_log_dir train_log \
                      BATCH_SIZE 4 \
                      SOLVER.LR 0.001

/home/aistudio/work/PaddleSeg02
/home/aistudio/work/PaddleSeg02
{'AUG': {'AUG_METHOD': 'unpadding',
         'FIX_RESIZE_SIZE': (512, 512),
         'FLIP': False,
         'FLIP_RATIO': 0.5,
         'INF_RESIZE_VALUE': 500,
         'MAX_RESIZE_VALUE': 600,
         'MAX_SCALE_FACTOR': 2.0,
         'MIN_RESIZE_VALUE': 400,
         'MIN_SCALE_FACTOR': 0.5,
         'MIRROR': True,
         'RICH_CROP': {'ASPECT_RATIO': 0.33,
                       'BLUR': False,
                       'BLUR_RATIO': 0.1,
                       'BRIGHTNESS_JITTER_RATIO': 0.5,
                       'CONTRAST_JITTER_RATIO': 0.5,
                       'ENABLE': False,
                       'MAX_ROTATION': 15,
                       'MIN_AREA_RATIO': 0.5,
                       'SATURATION_JITTER_RATIO': 0.5},
         'SCALE_STEP_SIZE': 0.25,
         'TO_RGB': False},
 'BATCH_SIZE': 4,
 'DATALOADER': {'BUF_SIZE': 256, 'NUM_WORKERS': 8},
 'DATASET': {'DATA_DIM': 3,
             'DATA_DIR': './dataset/

# 3. Model Evaluation

acc=0.9954 IoU=0.8720

In [3]:
# model evaluation
!python pdseg/eval.py --use_gpu \
                     --cfg configs/unet_optic.yaml \
                     TEST.TEST_MODEL saved_model/unet_optic/final

{'AUG': {'AUG_METHOD': 'unpadding',
         'FIX_RESIZE_SIZE': (512, 512),
         'FLIP': False,
         'FLIP_RATIO': 0.5,
         'INF_RESIZE_VALUE': 500,
         'MAX_RESIZE_VALUE': 600,
         'MAX_SCALE_FACTOR': 2.0,
         'MIN_RESIZE_VALUE': 400,
         'MIN_SCALE_FACTOR': 0.5,
         'MIRROR': True,
         'RICH_CROP': {'ASPECT_RATIO': 0.33,
                       'BLUR': False,
                       'BLUR_RATIO': 0.1,
                       'BRIGHTNESS_JITTER_RATIO': 0.5,
                       'CONTRAST_JITTER_RATIO': 0.5,
                       'ENABLE': False,
                       'MAX_ROTATION': 15,
                       'MIN_AREA_RATIO': 0.5,
                       'SATURATION_JITTER_RATIO': 0.5},
         'SCALE_STEP_SIZE': 0.25,
         'TO_RGB': False},
 'BATCH_SIZE': 4,
 'DATALOADER': {'BUF_SIZE': 256, 'NUM_WORKERS': 8},
 'DATASET': {'DATA_DIM': 3,
             'DATA_DIR': './dataset/optic_disc_seg/',
             'IGNORE_INDEX': 255,
            

# 4. Result

![](https://ai-studio-static-online.cdn.bcebos.com/7139a58cf72d4faabc2676229badfd3c3c2a514dc3404b2293926d9f93bb7b67)
![](https://ai-studio-static-online.cdn.bcebos.com/e2d22b17c66e4436b79b1d64292c242537a679902d8a4c0d99d978f65f17e075)


In [4]:
# visualize the model using vis.py
!python pdseg/vis.py --use_gpu \
                     --cfg configs/unet_optic.yaml \
                     TEST.TEST_MODEL saved_model/unet_optic/final

{'AUG': {'AUG_METHOD': 'unpadding',
         'FIX_RESIZE_SIZE': (512, 512),
         'FLIP': False,
         'FLIP_RATIO': 0.5,
         'INF_RESIZE_VALUE': 500,
         'MAX_RESIZE_VALUE': 600,
         'MAX_SCALE_FACTOR': 2.0,
         'MIN_RESIZE_VALUE': 400,
         'MIN_SCALE_FACTOR': 0.5,
         'MIRROR': True,
         'RICH_CROP': {'ASPECT_RATIO': 0.33,
                       'BLUR': False,
                       'BLUR_RATIO': 0.1,
                       'BRIGHTNESS_JITTER_RATIO': 0.5,
                       'CONTRAST_JITTER_RATIO': 0.5,
                       'ENABLE': False,
                       'MAX_ROTATION': 15,
                       'MIN_AREA_RATIO': 0.5,
                       'SATURATION_JITTER_RATIO': 0.5},
         'SCALE_STEP_SIZE': 0.25,
         'TO_RGB': False},
 'BATCH_SIZE': 4,
 'DATALOADER': {'BUF_SIZE': 256, 'NUM_WORKERS': 8},
 'DATASET': {'DATA_DIM': 3,
             'DATA_DIR': './dataset/optic_disc_seg/',
             'IGNORE_INDEX': 255,
            

请点击[此处](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576)查看本环境基本用法.  <br>
Please click [here ](https://ai.baidu.com/docs#/AIStudio_Project_Notebook/a38e5576) for more detailed instructions. 